In [1]:
import numpy as np
import matplotlib.pyplot as plt

n =  10000
eps  = 0.1
x = np.random.randn(n,2)
w = np.random.randn(2)
y = np.dot(x,w) + eps * np.random.randn(n)

In [2]:
w_hat = np.zeros(2)

def pred(x,w_hat):
    return np.dot(x,w_hat)
def loss(w_hat):
    return np.mean((y-pred(x,w_hat))**2)

In [3]:
epochs = 10
batch_size = 100
lr = 0.01


def train(w_hat, update):
    para_history = []
    loss_history = []

    para_history.append(w_hat)
    loss_history.append(loss(w_hat))
    for i in epochs:
        w_hat = update(w_hat)
    return para_history, loss_history



def grad(w_hat,idx=range(x.shape[0])):
    return np.dot(x[idx,:].T, pred(x[idx,:],w_hat)-y[idx])/len(idx)
def gradient_descent(w_hat):
    return w_hat - lr * grad(w_hat)
def mini_batch_gradient_descent(w_hat):
    shuffle = np.random.permutation(n)
    for i in range(0,n,batch_size):
        idx = shuffle[i:i+batch_size]
        w_hat - lr * grad(idx,w_hat)
    return w_hat
def stochastic_gradient_descent(w_hat):
    shuffle = np.random.permutation(n)
    for i in shuffle:
        w_hat - lr * grad(i,w_hat)
    return w_hat

In [5]:
grad(w_hat)

array([-0.6734998 ,  0.00537482])